In [1]:
!pip install transformers

     |████████████████████████████████| 1.3MB 4.9MB/s 
     |████████████████████████████████| 890kB 29.8MB/s 
     |████████████████████████████████| 2.9MB 38.3MB/s 
     |████████████████████████████████| 1.1MB 44.7MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893257 sha256=a6ddb76d2b29483880c2c318b62525116cbfc7e280b8a238a9cc49f45435be00
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


In [16]:
import time
import datetime
import random
from tqdm import tqdm

import numpy as np
import pandas as pd
import json

import nltk
nltk.download('punkt')

import torch
import torch.nn.functional as F
import transformers as optimus
from transformers import AdamW, get_linear_schedule_with_warmup

import torch.optim as optim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
from torch.utils.data import DataLoader, TensorDataset
from torch.utils.data.sampler import RandomSampler

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
tokenizer_class, pretrained_weights = (
    optimus.BertTokenizer,
    "bert-base-uncased",
)

# load the tokenizer from pre-trained model
tokenizer = tokenizer_class.from_pretrained(pretrained_weights, do_lower_case=True)

In [6]:
model_class, pretrained_weights = (
    optimus.BertForMultipleChoice,
    "bert-base-uncased",
)

# loading pre-trained bert model
model = model_class.from_pretrained(pretrained_weights, return_dict=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

In [7]:
def translateCSV(input_path, output_path):
    data = []
    with open(input_path) as f:
        for line in f:
            data.append(json.loads(line))

    df = pd.DataFrame(data)
    translatedCSV = { "premise":[], "choice1":[], "choice2": [], "question": [], "label": []}

    for index, row in df.iterrows():
        translatedCSV["premise"].append(row["premise"])
        translatedCSV["choice1"].append(row["choice1"])
        translatedCSV["choice2"].append(row["choice2"])
        translatedCSV["question"].append(row["question"])
        translatedCSV["label"].append(row["label"])

        if index%100 == 0:
            print(f'Finished {index} rows!!')

    df = pd.DataFrame(data=translatedCSV)
    df.to_csv(output_path, index=False)
    
translateCSV('./train.jsonl', './en-train.csv')
translateCSV('./val.jsonl', './en-val.csv')
# translateCSV('./test.jsonl', './en-test.csv')

Finished 0 rows!!
Finished 100 rows!!
Finished 200 rows!!
Finished 300 rows!!
Finished 0 rows!!


In [70]:
train_data = pd.read_csv('./en-train.csv')
val_data = pd.read_csv('./en-val.csv')

# val_data = pd.read_csv('./translated-gu-val.csv')
# val_data = pd.read_csv('./translated-hi-val.csv')
# val_data = pd.read_csv('./translated-mr-val.csv')

# val_data = pd.read_csv('./en-val.csv')
val_data

,premise,choice1,choice2,question,label
0,The man turned on the faucet.,The toilet filled with water.,Water flowed from the spout.,effect,1
1,The girl found a bug in her cereal.,She poured milk in the bowl.,She lost her appetite.,effect,1
2,The woman retired.,She received her pension.,She paid off her mortgage.,effect,0
3,I wanted to conserve energy.,I swept the floor in the unoccupied room.,I shut off the light in the unoccupied room.,effect,1
4,The hamburger meat browned.,The cook froze it.,The cook grilled it.,cause,1
...,...,...,...,...,...
95,My ears were ringing.,I went to a museum.,I went to a concert.,cause,1
96,I tidied up my house.,I was swamped with work.,I was expecting company.,cause,1
97,The airline mishandled my luggage.,They offered me compensation.,They cancelled my flight.,effect,0
98,The computer was expensive to fix.,I got it repaired.,I bought a new one.,effect,1


In [66]:
max_seq_len = 256
batch_size = 16

In [56]:
# initialize empty dataframe
train = pd.DataFrame()

# iterate each row in previously loaded train_data
# tokenizer(sent_a, sent_b) returns a dictionary of encoded values namely input_ids, attention_mask, token_type_ids
# input_ids - IDs returned by bert-tokenizer
# attention_mask - 1's for all the ids in the sentence and 0's for all padded tokens
# token_type_ids - 0's for all tokens in sent_a, 1's for all tokens in sent_b
for i, row in train_data.iterrows():
  d = {}
  q = row['question']
  encoded = tokenizer(
      [
       [row['premise']+f' What is the {q} of this?', row['premise']+f' What is the {q} of this?'],
       [row['choice1'], row['choice2']]
      ]
  )
  d['input_ids'] = encoded['input_ids']
  d['attention_mask'] = encoded['attention_mask']
  d['token_type_ids'] = encoded['token_type_ids']
  d['label'] = row['label']
  train = train.append(d, ignore_index=True)
  
# # padding all lists to MAX_SEQ_LENGTH
train['input_ids'] = train['input_ids'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])
train['attention_mask'] = train['attention_mask'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])
train['token_type_ids'] = train['token_type_ids'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])

In [14]:
# convert all lists to pytorch tensors
input_ids = torch.tensor([r for r in train['input_ids']], dtype=torch.long)
attention_mask = torch.tensor([r for r in train['attention_mask']], dtype=torch.long)
token_type_ids = torch.tensor([r for r in train['token_type_ids']], dtype=torch.long)
label_ids = torch.tensor([r for r in train['label']], dtype=torch.long)

# wraping tensors into TensorDataset
# Initialing DataLoader for efficiently loading data
train_dataset = TensorDataset(input_ids, attention_mask, token_type_ids, label_ids)
train_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [71]:
# initialize empty dataframe
val = pd.DataFrame()

# iterate each row in previously loaded train_data
# tokenizer(sent_a, sent_b) returns a dictionary of encoded values namely input_ids, attention_mask, token_type_ids
# input_ids - IDs returned by bert-tokenizer
# attention_mask - 1's for all the ids in the sentence and 0's for all padded tokens
# token_type_ids - 0's for all tokens in sent_a, 1's for all tokens in sent_b
for i, row in val_data.iterrows():
  d = {}
  q = row['question']
  encoded = tokenizer(
      [
       [row['premise']+f' What is the {q} of this?', row['premise']+f' What is the {q} of this?'],
       [row['choice1'], row['choice2']]
      ]
  )
  d['input_ids'] = encoded['input_ids']
  d['attention_mask'] = encoded['attention_mask']
  d['token_type_ids'] = encoded['token_type_ids']
  d['label'] = row['label']
  val = val.append(d, ignore_index=True)
  
# # padding all lists to MAX_SEQ_LENGTH
val['input_ids'] = val['input_ids'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])
val['attention_mask'] = val['attention_mask'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])
val['token_type_ids'] = val['token_type_ids'].apply(lambda x: [x[0] + (max_seq_len - len(x[0]))*[0], x[1] + (max_seq_len - len(x[1]))*[0]])

In [72]:
# convert all lists to pytorch tensors
input_ids = torch.tensor([r for r in val['input_ids']], dtype=torch.long)
attention_mask = torch.tensor([r for r in val['attention_mask']], dtype=torch.long)
token_type_ids = torch.tensor([r for r in val['token_type_ids']], dtype=torch.long)
label_ids = torch.tensor([r for r in val['label']], dtype=torch.long)

# wraping tensors into TensorDataset
# Initialing DataLoader for efficiently loading data
val_dataset = TensorDataset(input_ids, attention_mask, token_type_ids, label_ids)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size)

In [18]:
optimizer = AdamW(model.parameters(),
                  lr=1e-5, 
                  eps=1e-8)
                  
epochs = 5

scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps=0,
                                            num_training_steps=len(train_dataloader)*epochs)


In [ ]:
import random
from tqdm import tqdm
import numpy as np

seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

model = model.to(device)
  
epochs = 5
for epoch in tqdm(range(1, epochs+1)):
    
    model.train()
    
    loss_train_total = 0

    progress_bar = tqdm(train_dataloader, desc='Epoch {:1d}'.format(epoch), leave=False, disable=False)
    for batch in progress_bar:

        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        input_ids, input_mask, segment_ids, label_ids = batch

        # load tensors to device
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)      

        outputs = model(input_ids=input_ids,
                    attention_mask=input_mask,
                    token_type_ids=segment_ids,
                    labels=label_ids)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()

        # torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
         
        
    torch.save(model.state_dict(), f'data_volume/finetuned_BERT_epoch_{epoch}.model')
        
    tqdm.write(f'\nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(train_dataloader)            
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(val_dataloader)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (Weighted): {val_f1}')

In [24]:
from sklearn.metrics import f1_score

def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [23]:
def evaluate(dataloader_val):

    model.eval()
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in dataloader_val:
        
        input_ids, input_mask, segment_ids, label_ids = batch

        # load tensors to device
        input_ids = input_ids.to(device)
        input_mask = input_mask.to(device)
        segment_ids = segment_ids.to(device)
        label_ids = label_ids.to(device)

        with torch.no_grad():        
            outputs = model(input_ids=input_ids,
                    attention_mask=input_mask,
                    token_type_ids=segment_ids,
                    labels=label_ids)
            
        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()

        logits = logits.detach().cpu().numpy()
        label_ids = label_ids.cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader_val) 
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
            
    return loss_val_avg, predictions, true_vals

In [51]:
model = optimus.BertForMultipleChoice.from_pretrained("bert-base-uncased",
                                                      return_dict=True)

model.to(device)

model.load_state_dict(torch.load('data_volume/finetuned_BERT_epoch_5.model', map_location=torch.device('cpu')))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMultipleChoice: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMultipleChoice were not initialized from the model checkpoint at bert-base-uncased and are newly

<All keys matched successfully>

In [73]:
_, predictions, true_vals = evaluate(val_dataloader)
val_f1 = f1_score_func(predictions, true_vals)

val_f1

0.7995198079231693

In [ ]:
en - 0.7995198079231693

In [ ]:
mr - 0.6255411255411256

In [ ]:
hi - 0.6942628688028663

In [ ]:
gu - 0.7169100637063576